# Experiment Results Analysis

This notebook extracts experiment data into the exact DataFrame format requested.

In [1]:
import json
import pandas as pd
from typing import Dict, List, Any

In [2]:
def load_experiment_results(file_path: str) -> Dict[str, Any]:
    """Load experiment results from JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def extract_experiment_dataframe(results: Dict[str, Any]) -> pd.DataFrame:
    """
    Extract experiment data into a DataFrame with the exact columns requested.
    
    Args:
        results: Experiment results dictionary
        
    Returns:
        DataFrame with columns: Rounds, #Agents, Prompt, Model own, Model other agents, 
        Temperature self, Initial Preference, Agreement?, Agreed Principle
    """
    
    # Extract basic experiment info
    general_info = results.get('general_information', {})
    agents = results.get('agents', [])
    
    # Get experiment configuration
    rounds = general_info.get('phase2_rounds', 20)  # Default if not found
    num_agents = len(agents)
    
    # Extract consensus information
    consensus_reached = general_info.get('consensus_reached', False)
    consensus_principle = general_info.get('consensus_principle', None)
    
    # Map principle codes to numbers (based on your example showing "3")
    principle_to_number = {
        'maximizing_floor': 1,
        'maximizing_average': 2, 
        'maximizing_average_with_floor': 3,
        'maximizing_average_with_range': 4
    }
    
    agreed_principle_num = principle_to_number.get(consensus_principle, None) if consensus_principle else None
    
    # Create rows for each agent
    rows = []
    for agent in agents:
        # Get agent's model
        agent_model = agent.get('model', 'Unknown')
        
        # Get other agents' models (excluding current agent)
        other_models = [a.get('model', 'Unknown') for a in agents if a.get('name') != agent.get('name')]
        
        # Get agent's initial preference (first choice)
        initial_preference = None
        if 'phase_1' in agent and 'initial_ranking' in agent['phase_1']:
            ranking_data = agent['phase_1']['initial_ranking'].get('ranking_result', {})
            rankings = ranking_data.get('rankings', [])
            
            # Find rank 1 (first choice)
            for ranking in rankings:
                if ranking.get('rank') == 1:
                    principle = ranking.get('principle', '')
                    initial_preference = principle_to_number.get(principle, None)
                    break
        
        row = {
            'Rounds': rounds,
            '#Agents': num_agents,
            'Prompt': agent.get('personality', ''),
            'Model own': agent_model,
            'Model other agents': other_models,
            'Temperature self': agent.get('temperature', 0),
            'Initial Preference': initial_preference,
            'Agreement?': 1 if consensus_reached else 0,  # 1 for Yes, 0 for No
            'Agreed Principle': agreed_principle_num
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

## Load Multiple JSON Files and Create Sankey Chart

This section loads all JSON experiment files from predefined folders, combines them, and creates a Sankey chart.

In [ ]:
import glob
import os
import plotly.graph_objects as go

# Define predefined folders to search for JSON files
PREDEFINED_FOLDERS = [
    "/Users/lucasmuller/Desktop/Githubg/Rawls_v3/",
    "/Users/lucasmuller/Desktop/Githubg/Rawls_v3/hypothesis_2_&_4/",
    "/Users/lucasmuller/Desktop/Githubg/Rawls_v3/hypothesis_2_&_4/logs/"
]

def load_multiple_experiments(folders: List[str] = PREDEFINED_FOLDERS) -> pd.DataFrame:
    """
    Load multiple experiment JSON files from predefined folders and combine them.
    
    Args:
        folders: List of folder paths to search for JSON files
        
    Returns:
        Combined DataFrame with all experiment data
    """
    
    all_files = []
    
    # Search for JSON files in all predefined folders
    for folder in folders:
        if os.path.exists(folder):
            # Look for experiment result files and config result files
            patterns = [
                os.path.join(folder, "experiment_results_*.json"),
                os.path.join(folder, "*_results_*.json"),
                os.path.join(folder, "config_*_results_*.json")
            ]
            
            for pattern in patterns:
                files = glob.glob(pattern)
                all_files.extend(files)
    
    # Remove duplicates
    all_files = list(set(all_files))
    
    if not all_files:
        print("No JSON experiment files found in predefined folders:")
        for folder in folders:
            print(f"  - {folder}")
        return pd.DataFrame()
    
    print(f"Found {len(all_files)} JSON experiment files:")
    
    # Load and combine all experiments
    all_dfs = []
    for file_path in all_files:
        try:
            results = load_experiment_results(file_path)
            df_single = extract_experiment_dataframe(results)
            df_single['Experiment_File'] = os.path.basename(file_path)
            all_dfs.append(df_single)
            print(f"✓ Loaded {os.path.basename(file_path)} ({len(df_single)} agents)")
        except Exception as e:
            print(f"✗ Error loading {os.path.basename(file_path)}: {e}")
    
    if not all_dfs:
        print("No valid experiment files could be loaded")
        return pd.DataFrame()
    
    # Combine all DataFrames
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"\n✅ Combined {len(combined_df)} total agent observations from {len(all_dfs)} experiments")
    
    return combined_df

def create_sankey_chart(df: pd.DataFrame, title: str = "Flow from Initial to Agreed Principle"):
    """
    Create a Sankey chart showing flow from Initial Preference to Agreed Principle.
    
    Args:
        df: DataFrame with Initial Preference and Agreement/Agreed Principle columns
        title: Chart title
    """
    
    if df.empty:
        print("Cannot create Sankey chart: DataFrame is empty")
        return None
    
    # Map principle numbers to names
    principle_names = {
        1: "Maximizing Floor",
        2: "Maximizing Average", 
        3: "Maximizing Average with Floor",
        4: "Maximizing Average with Range"
    }
    
    # Create source and target labels
    source_labels = []
    target_labels = []
    values = []
    
    # Group by Initial Preference and outcome
    for _, row in df.iterrows():
        initial = row['Initial Preference']
        agreement = row['Agreement?']
        agreed = row['Agreed Principle']
        
        # Source: Initial Preference
        initial_name = principle_names.get(initial, f"Unknown ({initial})")
        source_name = f"Initial: {initial_name}"
        
        # Target: Either agreed principle or "No Agreement"
        if agreement == 1 and agreed is not None:
            target_name = f"Agreed: {principle_names.get(agreed, f'Unknown ({agreed})')}"
        else:
            target_name = "No Agreement"
        
        source_labels.append(source_name)
        target_labels.append(target_name)
        values.append(1)  # Each agent contributes 1 to the flow
    
    # Create unique labels and map to indices
    all_labels = list(set(source_labels + target_labels))
    label_to_index = {label: i for i, label in enumerate(all_labels)}
    
    # Convert to indices and aggregate flows
    flows = {}
    for source, target, value in zip(source_labels, target_labels, values):
        source_idx = label_to_index[source]
        target_idx = label_to_index[target]
        key = (source_idx, target_idx)
        flows[key] = flows.get(key, 0) + value
    
    # Extract final data
    source_indices = []
    target_indices = []
    flow_values = []
    
    for (source_idx, target_idx), value in flows.items():
        source_indices.append(source_idx)
        target_indices.append(target_idx)
        flow_values.append(value)
    
    # Create colors
    colors = []
    for label in all_labels:
        if "Initial:" in label:
            colors.append("rgba(31, 119, 180, 0.8)")  # Blue for initial
        elif "No Agreement" in label:
            colors.append("rgba(214, 39, 40, 0.8)")   # Red for no agreement
        else:
            colors.append("rgba(44, 160, 44, 0.8)")   # Green for agreed
    
    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_labels,
            color=colors
        ),
        link=dict(
            source=source_indices,
            target=target_indices,
            value=flow_values,
            color="rgba(128, 128, 128, 0.4)"
        )
    )])
    
    fig.update_layout(
        title_text=title,
        font_size=12,
        height=600,
        width=1000
    )
    
    return fig

# Load all experiments from predefined folders and create Sankey chart
combined_df = load_multiple_experiments()

if not combined_df.empty:
    # Display summary statistics
    print("\n=== Summary Statistics ===")
    print(f"Total Agents: {len(combined_df)}")
    print(f"Agents with Agreement: {combined_df['Agreement?'].sum()}")
    print(f"Agents without Agreement: {len(combined_df) - combined_df['Agreement?'].sum()}")
    print(f"Agreement Rate: {(combined_df['Agreement?'].sum() / len(combined_df) * 100):.1f}%")
    
    print("\nInitial Preference Distribution:")
    initial_counts = combined_df['Initial Preference'].value_counts().sort_index()
    principle_names = {1: "Maximizing Floor", 2: "Maximizing Average", 
                      3: "Maximizing Average with Floor", 4: "Maximizing Average with Range"}
    for principle, count in initial_counts.items():
        print(f"  {principle_names.get(principle, f'Unknown ({principle})')}: {count}")
    
    print("\nExperiment Files:")
    file_counts = combined_df['Experiment_File'].value_counts()
    for file, count in file_counts.items():
        print(f"  {file}: {count} agents")
    
    # Create and display Sankey chart
    sankey_fig = create_sankey_chart(
        combined_df, 
        title=f"Flow from Initial to Agreed Principle ({len(file_counts)} Experiments, {len(combined_df)} Agents)"
    )
    
    if sankey_fig:
        sankey_fig.show()
        print("\n✅ Sankey chart created successfully!")
    else:
        print("❌ Failed to create Sankey chart")
        
    # Display the combined DataFrame
    print(f"\nCombined DataFrame ({len(combined_df)} rows):")
    display(combined_df.head(10))
else:
    print("❌ No experiment data loaded - cannot create Sankey chart")

In [6]:
import glob
import os

def analyze_multiple_experiments(results_pattern: str = "/Users/lucasmuller/Desktop/Githubg/Rawls_v3/experiment_results_*.json"):
    """
    Analyze multiple experiment files and create combined Sankey chart.
    
    Args:
        results_pattern: Glob pattern to match experiment result files
    """
    
    # Find all matching files
    files = glob.glob(results_pattern)
    if not files:
        print(f"No files found matching pattern: {results_pattern}")
        return None
    
    print(f"Found {len(files)} experiment files")
    
    # Combine all experiments into one DataFrame
    all_dfs = []
    for file_path in files:
        try:
            results = load_experiment_results(file_path)
            df_single = extract_experiment_dataframe(results)
            df_single['Experiment_File'] = os.path.basename(file_path)
            all_dfs.append(df_single)
            print(f"✓ Loaded {file_path}")
        except Exception as e:
            print(f"✗ Error loading {file_path}: {e}")
    
    if not all_dfs:
        print("No valid experiment files could be loaded")
        return None
    
    # Combine all DataFrames
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"\nCombined data: {len(combined_df)} total agent observations from {len(all_dfs)} experiments")
    
    # Create Sankey chart for combined data
    sankey_fig = create_sankey_chart(
        combined_df, 
        title=f"Flow from Initial to Agreed Principle (Combined Analysis - {len(all_dfs)} Experiments)"
    )
    
    # Display summary statistics
    print("\n=== Summary Statistics ===")
    print(f"Total Agents: {len(combined_df)}")
    print(f"Experiments with Agreement: {combined_df['Agreement?'].sum()}")
    print(f"Experiments without Agreement: {len(combined_df) - combined_df['Agreement?'].sum()}")
    
    print("\nInitial Preference Distribution:")
    initial_counts = combined_df['Initial Preference'].value_counts().sort_index()
    principle_names = {1: "Maximizing Floor", 2: "Maximizing Average", 
                      3: "Maximizing Average with Floor", 4: "Maximizing Average with Range"}
    for principle, count in initial_counts.items():
        print(f"  {principle_names.get(principle, f'Unknown ({principle})')}: {count}")
    
    return combined_df, sankey_fig

# Run batch analysis if experiment files exist
try:
    combined_df, batch_sankey = analyze_multiple_experiments()
    if batch_sankey:
        batch_sankey.show()
except Exception as e:
    print(f"Batch analysis error: {e}")
    print("This is normal if you don't have multiple experiment files yet.")

Found 2 experiment files
✓ Loaded /Users/lucasmuller/Desktop/Githubg/Rawls_v3/experiment_results_20250810_190409.json
✓ Loaded /Users/lucasmuller/Desktop/Githubg/Rawls_v3/experiment_results_20250810_223209.json

Combined data: 6 total agent observations from 2 experiments
Batch analysis error: name 'create_sankey_chart' is not defined
This is normal if you don't have multiple experiment files yet.


### Batch Analysis: Multiple Experiment Files

You can also create Sankey charts for multiple experiment files to see patterns across different experimental conditions.

In [7]:
%pip install  nbformat 


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import nbformat


In [9]:
import plotly.graph_objects as go
import plotly.express as px
import nbformat
from collections import Counter

def create_sankey_chart(df: pd.DataFrame, title: str = "Flow from Initial to Agreed Principle"):
    """
    Create a Sankey chart showing flow from Initial Preference to Agreed Principle.
    
    Args:
        df: DataFrame with Initial Preference and Agreement/Agreed Principle columns
        title: Chart title
    """
    
    # Map principle numbers to names
    principle_names = {
        1: "Maximizing Floor",
        2: "Maximizing Average", 
        3: "Maximizing Average with Floor",
        4: "Maximizing Average with Range"
    }
    
    # Create source and target labels
    source_labels = []
    target_labels = []
    values = []
    
    # Group by Initial Preference and outcome
    for _, row in df.iterrows():
        initial = row['Initial Preference']
        agreement = row['Agreement?']
        agreed = row['Agreed Principle']
        
        # Source: Initial Preference
        initial_name = principle_names.get(initial, f"Unknown ({initial})")
        source_name = f"Initial: {initial_name}"
        
        # Target: Either agreed principle or "No Agreement"
        if agreement == 1 and agreed is not None:
            target_name = f"Agreed: {principle_names.get(agreed, f'Unknown ({agreed})')}"
        else:
            target_name = "No Agreement"
        
        source_labels.append(source_name)
        target_labels.append(target_name)
        values.append(1)  # Each agent contributes 1 to the flow
    
    # Create unique labels and map to indices
    all_labels = list(set(source_labels + target_labels))
    label_to_index = {label: i for i, label in enumerate(all_labels)}
    
    # Convert to indices and aggregate flows
    flows = {}
    for source, target, value in zip(source_labels, target_labels, values):
        source_idx = label_to_index[source]
        target_idx = label_to_index[target]
        key = (source_idx, target_idx)
        flows[key] = flows.get(key, 0) + value
    
    # Extract final data
    source_indices = []
    target_indices = []
    flow_values = []
    
    for (source_idx, target_idx), value in flows.items():
        source_indices.append(source_idx)
        target_indices.append(target_idx)
        flow_values.append(value)
    
    # Create colors
    colors = []
    for label in all_labels:
        if "Initial:" in label:
            colors.append("rgba(31, 119, 180, 0.8)")  # Blue for initial
        elif "No Agreement" in label:
            colors.append("rgba(214, 39, 40, 0.8)")   # Red for no agreement
        else:
            colors.append("rgba(44, 160, 44, 0.8)")   # Green for agreed
    
    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_labels,
            color=colors
        ),
        link=dict(
            source=source_indices,
            target=target_indices,
            value=flow_values,
            color="rgba(128, 128, 128, 0.4)"
        )
    )])
    
    fig.update_layout(
        title_text=title,
        font_size=12,
        height=600,
        width=1000
    )
    
    return fig

# Create and display the Sankey chart
# First, let's load some data to demonstrate
try:
    # Try to use existing df if available, otherwise load fresh data
    if 'df' not in locals():
        results = load_experiment_results("/Users/lucasmuller/Desktop/Githubg/Rawls_v3/hypothesis_2_&_4/config_01_results_20250810_094637.json")
        df = extract_experiment_dataframe(results)
    
    sankey_fig = create_sankey_chart(df)
    sankey_fig.show()
    
except Exception as e:
    print(f"Error creating Sankey chart: {e}")
    print("Make sure you have plotly installed: pip install plotly")

## Sankey Chart: Initial to Agreed Principle

This section creates a Sankey diagram showing the flow from agents' initial preferences to the final agreed principle.